In [1]:
from typing import List, Tuple, Union
import numpy as np

In [2]:
BLACK	= '\033[30m'
RED		= '\033[31m'
GREEN	= '\033[32m'
YELLOW	= '\033[33m'
BLUE	= '\033[34m'
MAGENTA	= '\033[35m'
CYAN	= '\033[36m'
RESET	= '\033[0m'
SEL		= '\033[7m'
SPACE   = ' '

In [162]:
# Two's Complement Format
class cls_signed_mult:
    def __init__(self, i_bits, i_radix, **kargs):
        self.i_bits     = i_bits
        self.i_radix    = i_radix
        self.min_val    = -1 * (2 ** (i_bits - 1))
        self.max_val    = 2 ** (i_bits - 1) - 1

        if 'in_a' in kargs.keys():
            self.i_in_a = kargs['in_a']
        else:
            self.i_in_a = np.random.randint(self.min_val, self.max_val, 1)[0]

        if 'in_b' in kargs.keys():
            self.i_in_b = kargs['in_b']
        else:
            self.i_in_b = np.random.randint(self.min_val, self.max_val, 1)[0]

        self.s_in_a = self.dec2bin(self.i_in_a, self.i_bits)
        self.s_in_b = self.dec2bin(self.i_in_b, self.i_bits)
        
        self.div_booth_enc_inputs()
        self.ppg_radix4_cmosvlsi()
        self.pps_fill_r4()
        self.disp()
        
    def dec2bin(self, i_dec: int, i_bits: int) -> str:
        if i_dec >= 0:
            s_bin = bin(i_dec)[2:].rjust(i_bits, '0')
        else:
            bit_flip = '1' + ''.join(str(1 ^ int(x)) for x in bin(abs(i_dec))[2:])
            plus_one = int(bit_flip, 2) + 1
            s_bin = bin(plus_one)[3:].rjust(i_bits, '1')
        return  s_bin

    def bin2dec(self, s_bin: str) -> int:
        i_dec = 0
        for idx, a_str in enumerate(s_bin[::-1]):
            if idx == len(s_bin) - 1:
                i_dec = i_dec - int(a_str) * (2 ** idx)
            else:
                i_dec = i_dec + int(a_str) * (2 ** idx)
        return i_dec

    def div_booth_enc_inputs(self):
        div = int(np.log2(self.i_radix))
        booth_enc_inputs = list()
        for i in range(int(np.ceil(self.i_bits/div))):
            booth_enc_input = ''.join(list('0' + self.s_in_b[::-1])[div*i:div*i+div+1])
            # sign extension for input of booth encoder
            if len(booth_enc_input) != div+1:
                booth_enc_input = booth_enc_input + (booth_enc_input[-1]) * (div+1 - len(booth_enc_input))
            booth_enc_inputs.append(booth_enc_input)
        # index: -1, 0, 1, ..., n-1
        self.booth_enc_inputs = booth_enc_inputs

    def booth_enc_radix4_cmosvlsi(self, booth_enc_input: str):
        x_in = list(map(int, list(booth_enc_input)))
        single  = x_in[0] ^ x_in[1]
        double  = int(not(not(x_in[0] and x_in[1] and not(x_in[2])) and not(not(x_in[0]) and not(x_in[1]) and x_in[2])))
        neg     = x_in[2]
        return ''.join(str(x) for x in [single, double, neg])

    def booth_sel_radix4_cmosvlsi(self, booth_enc_out: str, Y: str):
        x_vals  = list(map(int, list(booth_enc_out)))
        s, d, n = x_vals
        y_vals  = [int(Y[0])] + list(map(int, list(Y)))
        y_shft  = list(map(int, list(Y))) + [0]
        pp      = []
        for y in list(zip(y_shft[::-1], y_vals[::-1])):
            pp.append(str(int(not(not(d and y[0]) and not(s and y[1])) ^ n)))
        pp = ''.join(pp[::-1])
        return pp, str(n)

    def ppg_radix4_cmosvlsi(self):
        self.pps    = []
        self.negs   = []
        for booth_enc_input in self.booth_enc_inputs:
            booth_enc_out = self.booth_enc_radix4_cmosvlsi(booth_enc_input)
            pp, n = self.booth_sel_radix4_cmosvlsi(booth_enc_out, self.s_in_a)
            self.pps.append(pp)
            self.negs.append(n)
    
    def pps_fill_r4(self):
        self.pps_fill = []
        for idx, ppneg in enumerate(list(zip(self.pps, self.negs))):
            pp  = ppneg[0]
            n   = ppneg[1]
            e   = pp[0]
            eb  = str(int(not(int(e))))
            if idx == 0:
                pp_fill = (self.i_bits-(2*idx+1)-3)*'0' + eb + 2*e + pp
            else:
                if idx != len(self.pps) - 1:
                    pp_fill = (self.i_bits-(2*idx+1)-2)*'0' + '1' + eb + pp + '0' + s + (2*(idx-1))*'0'
                else:
                    pp_fill = eb + pp + '0' + s + 2*(idx-1)*'0'
            s   = n
            self.pps_fill.append(pp_fill)
        last_line = (s + 2*(len(self.pps)-1)*'0').zfill(2*self.i_bits)
        self.pps_fill.append(last_line)

    def disp(self):
        print(f'{self.booth_enc_inputs}')
        print(f' {SPACE:>{self.i_bits}}{self.s_in_a:>{self.i_bits}} ({self.i_in_a})')
        print(f'x{SPACE:>{self.i_bits}}{self.s_in_b:>{self.i_bits}} ({self.i_in_b})')
        print((self.i_bits*2+1)*'-')
        for idx, ppneg in enumerate(list(zip(self.pps, self.negs))):
            pp = ppneg[0]
            n = ppneg[1]
            if idx == 0:
                print(f'{SPACE:>{self.i_bits - 2*idx - 3}}{YELLOW}{int(not(int(pp[0])))}{2*pp[0]}{BLUE}{pp}')
            if 0 < idx < len(self.pps) - 1:
                print(f'{SPACE:>{self.i_bits - 2*idx - 2}}{YELLOW}{1}{int(not(int(pp[0])))}{BLUE}{pp}{SPACE}{RED}{s}')
            if idx == len(self.pps) - 1:
                print(f'{SPACE:>{self.i_bits - 2*idx - 1}}{YELLOW}{int(not(int(pp[0])))}{BLUE}{pp}{SPACE}{RED}{s}')
            s = n
        print(f'{SPACE:>{10}}{RED}{s}{RESET}')

        result = 0
        for pp_fill in self.pps_fill:
            print(f' {pp_fill}')
            result = result + self.bin2dec(pp_fill)
        print((self.i_bits*2+1)*'-')
        # Overflow Handling
        if result > 2**16-1:
            result = result - 2**16
        result = self.dec2bin(result, 16)
        print(f' {RED}{result}')
        print(f' {GREEN}{self.dec2bin(self.i_in_a*self.i_in_b, 16)}{RESET}')
        result = self.bin2dec(result)
        print(self.i_in_a*self.i_in_b, result)

In [164]:
test_cls = cls_signed_mult(8, 4)

['010', '000', '010', '000']
         01000100 (68)
x        00010001 (17)
-----------------
     100001000100
    11000000000 0
  11001000100 0
 1000000000 0
          0
 0000100001000100
 0001100000000000
 0110010001000000
 1000000000000000
 0000000000000000
-----------------
 0000010010000100
 0000010010000100
1156 1156


![](imgs/2023-07-17-20-14-17.png)

![](imgs/2023-07-18-07-34-15.png)